This notebook follows from the tutorials of CHGNet to relax LLZO-Li slabs that have been made in `llzo_li_balanced_sliced` directory.


LLZO‖Li Interface Relaxation Notebook
========================================

Each notebook handles only one structure.
1. Purpose: (This keeps updating)
- Relax a single LLZO‖Li (in this notebook LLZO_001_Zr_code93_sto__Li_110_slab_heavy) heterostructure using CHGNet
- Perform multi-stage optimization (in this notebook, CG → FIRE)
- Freeze bulk-like regions (15 Å at both ends)
- after all this, Relax lattice vectors to relieve interface strain

2. This notebook handles:
- Structure: LLZO_110_Li_order17_off__Li_100_slab_heavy
- Initial lattice height: 86.02 Å
- Number of atoms: 738

3. Method:
- CHGNet (v0.4.0) + ASE interface
- Stage 1: SciPyFminCG (no cell relaxation) → fmax target ~0.15 eV/Å
- Stage 2: FIRE (with optional cell relaxation) → fmax target ~0.05 eV/Å
- FrechetCellFilter used for combined force + stress minimization

4. Constraints:
- LLZO base: frozen bottom 14.25 Å
- Li top: frozen top 14.25 Å
- Only interfacial region relaxed
- Cell relaxation via `relax_cell=True` and `relax_cell_atoms="unconstrained"`

5. Outputs: (This will be decided later)
- relaxed_[structure_name].cif
- relaxed_[structure_name].traj
- (Optional) relaxation_log.pkl with energies, forces

6. Visual checks:
- Compare pre- and post-relaxation structures
- Ensure no Li diffusion into LLZO (via z-analysis)
- Confirm convergence (fmax < 0.05 eV/Å)

Author: Mehul Darak

Date: 15-07-2025


In [1]:
structure_name = "LLZO_010_Li_order0_off__Li_100_slab_heavy"

In [2]:
from pymatgen.core import Structure
import os

# Load structure
structure_path = (f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")  # replace with your file
structure = Structure.from_file(structure_path)

# Extract info
structure_name = os.path.basename(structure_path).replace(".cif", "")
lattice_height = structure.lattice.c
num_atoms = len(structure)

# Print output
print(f"- Structure: {structure_name}")
print(f"- Initial lattice height: {lattice_height:.2f} Å")
print(f"- Number of atoms: {num_atoms}")


- Structure: LLZO_010_Li_order0_off__Li_100_slab_heavy
- Initial lattice height: 69.56 Å
- Number of atoms: 584


In [3]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 21.41 Å
Lowest Li slab atom: 33.16 Å
LLZO top z: 32.79 Å
Li penetration into LLZO: -0.36 Å


In [4]:
from pymatgen.core import Structure
import numpy as np

# Load structure
structure = Structure.from_file(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# Get z-coordinates and element types
z_coords = np.array([site.z for site in structure.sites])
species = np.array([site.species_string for site in structure.sites])

# LLZO: non-Li atoms (La, Zr, O)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()
llzo_bottom = llzo_z.min()
llzo_thickness = llzo_top - llzo_bottom

# Li slab: Li atoms ABOVE LLZO (i.e. in metallic Li layer)
li_slab_z = np.array([
    site.z for site in structure.sites
    if site.species_string == "Li" and site.z > llzo_top
])
li_thickness = li_slab_z.ptp() if len(li_slab_z) > 0 else 0
li_bottom = li_slab_z.min() if len(li_slab_z) > 0 else None

# Penetration check
penetration = llzo_top - li_bottom if li_bottom is not None else 0

# Report
print(f"LLZO slab thickness: {llzo_thickness:.2f} Å")
print(f"Li slab thickness:   {li_thickness:.2f} Å")
print(f"LLZO top z:          {llzo_top:.2f} Å")
print(f"Lowest Li atom z:    {li_bottom:.2f} Å" if li_bottom else "No Li slab atoms found")
print(f"Li penetration into LLZO: {penetration:.2f} Å")


LLZO slab thickness: 17.39 Å
Li slab thickness:   21.41 Å
LLZO top z:          32.79 Å
Lowest Li atom z:    33.16 Å
Li penetration into LLZO: -0.36 Å


In [5]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.constraints import FixAtoms
from chgnet.model.dynamics import CHGNetCalculator, StructOptimizer
from ase.io import read, write
import numpy as np

# --- Load structure ---
structure = read(f"/home/mehuldarak/summer/llzo_li_balanced_sliced/{structure_name}.cif")

# --- Get z coordinates ---
z_coords = structure.get_positions()[:, 2]
z_min, z_max = z_coords.min(), z_coords.max()

# --- Define freeze zones ---
freeze_thickness_llzo = 0.75 * (llzo_thickness)  # in Å
freeze_thickness_li = 0.75 * (llzo_thickness)  # in Å
llzo_z_threshold = z_min + freeze_thickness_llzo
li_z_threshold = z_max - freeze_thickness_li

# --- Freeze LLZO base and Li top ---
freeze_mask = (z_coords < llzo_z_threshold) | (z_coords > li_z_threshold)
structure.set_constraint(FixAtoms(mask=freeze_mask))
print(f"Freezing {np.sum(freeze_mask)} atoms out of {len(structure)}")

# --- Attach CHGNet calculator ---
calc = CHGNetCalculator(use_device="cuda")
structure.set_calculator(calc)

Freezing 402 atoms out of 584
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda


/tmp/ipykernel_169442/3870098642.py:27: FutureWarning: Please use atoms.calc = calc
  structure.set_calculator(calc)


In [6]:
# Stage 1: CG
opt1 = StructOptimizer(model=calc, optimizer_class="SciPyFminCG", use_device="cuda")
result1 = opt1.relax(structure, fmax=0.15, steps=300, relax_cell=False, verbose=True)

             Step     Time          Energy          fmax
SciPyFminCG:    0 18:10:14    -2238.711315        0.601346
SciPyFminCG:    1 18:10:15    -2242.534470        5.074744
SciPyFminCG:    2 18:10:17    -2249.793423        5.984256
SciPyFminCG:    3 18:10:17    -2254.559069        3.507169
SciPyFminCG:    4 18:10:18    -2266.667070        3.411249
SciPyFminCG:    5 18:10:20    -2282.755564        3.736096
SciPyFminCG:    6 18:10:21    -2287.459110        4.148628
SciPyFminCG:    7 18:10:21    -2289.824181        2.262356
SciPyFminCG:    8 18:10:22    -2290.596664        0.530682
SciPyFminCG:    9 18:10:23    -2290.921225        0.696107
SciPyFminCG:   10 18:10:25    -2291.968979        1.656156
SciPyFminCG:   11 18:10:25    -2292.711527        1.980707
SciPyFminCG:   12 18:10:26    -2295.904358        2.325833
SciPyFminCG:   13 18:10:27    -2299.857977        4.777312
SciPyFminCG:   14 18:10:28    -2307.048704        5.371006
SciPyFminCG:   15 18:10:29    -2316.256271        4.145487

In [7]:
# Convert back, assign calculator + constraint
structure_1 = AseAtomsAdaptor.get_atoms(result1["final_structure"])
structure_1.set_calculator(calc)
structure_1.set_constraint(FixAtoms(mask=freeze_mask))

# Stage 2: FIRE
opt2 = StructOptimizer(model=calc, optimizer_class="FIRE", use_device="cuda")
result2 = opt2.relax(structure_1, fmax=0.05, steps=400, relax_cell=False, verbose=True)

/tmp/ipykernel_169442/3154571052.py:3: FutureWarning: Please use atoms.calc = calc
  structure_1.set_calculator(calc)


      Step     Time          Energy          fmax
FIRE:    0 18:11:55    -2375.686844        0.131068
FIRE:    1 18:11:56    -2375.691021        0.117376
FIRE:    2 18:11:56    -2375.698818        0.115685
FIRE:    3 18:11:57    -2375.708843        0.114786
FIRE:    4 18:11:57    -2375.720261        0.114520
FIRE:    5 18:11:57    -2375.732235        0.115132
FIRE:    6 18:11:58    -2375.744766        0.113919
FIRE:    7 18:11:58    -2375.757576        0.121343
FIRE:    8 18:11:58    -2375.771221        0.133703
FIRE:    9 18:11:59    -2375.786537        0.143695
FIRE:   10 18:11:59    -2375.804081        0.151835
FIRE:   11 18:12:00    -2375.825523        0.154803
FIRE:   12 18:12:00    -2375.851978        0.143258
FIRE:   13 18:12:00    -2375.884560        0.129011
FIRE:   14 18:12:01    -2375.923267        0.114387
FIRE:   15 18:12:01    -2375.966431        0.098875
FIRE:   16 18:12:02    -2376.012100        0.087337
FIRE:   17 18:12:02    -2376.059441        0.086845
FIRE:   18 18:

In [8]:
from pymatgen.io.ase import AseAtomsAdaptor
from ase.io import write

# Extract final structure from result3 (FIRE)
final_structure_pmg = result2["final_structure"]  # assuming result2 = FIRE
final_structure_ase = AseAtomsAdaptor.get_atoms(final_structure_pmg)

# Save as CIF and ASE trajectory
write(f"relaxed_{structure_name}.cif", final_structure_ase)
write(f"relaxed_{structure_name}.traj", final_structure_ase)

print("✅ Final structure saved successfully.")

✅ Final structure saved successfully.


In [9]:
from pymatgen.core import Structure
import numpy as np

s = Structure.from_file(f"/home/mehuldarak/summer/relax_final/{structure_name}/relaxed_{structure_name}.cif")

# Get all atoms
z_coords = np.array([site.z for site in s.sites])
species = np.array([site.species_string for site in s.sites])

# Estimate LLZO top (non-Li atoms)
llzo_z = z_coords[species != "Li"]
llzo_top = llzo_z.max()

# Now isolate Li slab: Li atoms ABOVE LLZO
li_slab_z = np.array([site.z for site in s.sites if site.species_string == "Li" and site.z > llzo_top])

print(f"Li slab thickness: {li_slab_z.ptp():.2f} Å")
print(f"Lowest Li slab atom: {li_slab_z.min():.2f} Å")
print(f"LLZO top z: {llzo_top:.2f} Å")
print(f"Li penetration into LLZO: {llzo_top - li_slab_z.min():.2f} Å")


Li slab thickness: 21.69 Å
Lowest Li slab atom: 32.87 Å
LLZO top z: 32.57 Å
Li penetration into LLZO: -0.30 Å


In [10]:
import os
from chgnet.model import StructOptimizer
from pymatgen.core import Structure
from chgnet.model.dynamics import CHGNetCalculator

structure_path = f"/home/mehuldarak/summer/relax_final/{structure_name}/relaxed_{structure_name}.cif"
structure = Structure.from_file(structure_path)

# Output path
output_dir = f"/home/mehuldarak/summer/relax_final"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, f"cellrelaxed_{structure_name}.cif")

# Run CHGNet relaxation
opt1 = StructOptimizer(model=calc, optimizer_class="SciPyFminCG", use_device="cuda")
result = opt1.relax(
    structure,
    fmax=0.15,           # You can adjust depending on accuracy/speed tradeoff
    steps=400,
    relax_cell=True,
    verbose=True
)

             Step     Time          Energy          fmax
SciPyFminCG:    0 18:12:13    -2376.442898        1.030956
SciPyFminCG:    1 18:12:14    -2378.156063        0.819470
SciPyFminCG:    2 18:12:15    -2380.286938        1.415608
SciPyFminCG:    3 18:12:16    -2381.120407        0.856374
SciPyFminCG:    4 18:12:17    -2382.125137        0.801900
SciPyFminCG:    5 18:12:18    -2383.067211        0.877017
SciPyFminCG:    6 18:12:19    -2383.963058        0.772334
SciPyFminCG:    7 18:12:20    -2384.586281        0.774304
SciPyFminCG:    8 18:12:21    -2385.346233        1.650154
SciPyFminCG:    9 18:12:22    -2386.268536        1.068720
SciPyFminCG:   10 18:12:23    -2388.173012        1.892425
SciPyFminCG:   11 18:12:24    -2391.035713        1.355736
SciPyFminCG:   12 18:12:24    -2392.570656        1.533744
SciPyFminCG:   13 18:12:25    -2393.368202        1.144328
SciPyFminCG:   14 18:12:26    -2393.753887        0.674325
SciPyFminCG:   15 18:12:27    -2394.184128        0.704792

In [11]:
# Convert back, assign calculator + constraint
structure_1 = AseAtomsAdaptor.get_atoms(result["final_structure"])
structure_1.set_calculator(calc)
structure_1.set_constraint(FixAtoms(mask=freeze_mask))

# Stage 2: FIRE
opt2 = StructOptimizer(model=calc, optimizer_class="FIRE", use_device="cuda")
result2 = opt2.relax(structure_1, fmax=0.05, steps=400, relax_cell=True, verbose=True)

/tmp/ipykernel_169442/1077038773.py:3: FutureWarning: Please use atoms.calc = calc
  structure_1.set_calculator(calc)


      Step     Time          Energy          fmax
FIRE:    0 18:14:49    -2476.801468        0.125716
FIRE:    1 18:14:49    -2476.802303        0.125702
FIRE:    2 18:14:50    -2476.803974        0.125678
FIRE:    3 18:14:50    -2476.806480        0.125648
FIRE:    4 18:14:50    -2476.809544        0.125615
FIRE:    5 18:14:51    -2476.812885        0.125581
FIRE:    6 18:14:51    -2476.816784        0.125549
FIRE:    7 18:14:51    -2476.820961        0.125517
FIRE:    8 18:14:52    -2476.826530        0.125478
FIRE:    9 18:14:52    -2476.833214        0.125417
FIRE:   10 18:14:52    -2476.841568        0.125317
FIRE:   11 18:14:53    -2476.851036        0.125137
FIRE:   12 18:14:53    -2476.862732        0.124852
FIRE:   13 18:14:53    -2476.876377        0.124428
FIRE:   14 18:14:53    -2476.892529        0.123854
FIRE:   15 18:14:54    -2476.910908        0.123206
FIRE:   16 18:14:54    -2476.931793        0.122516
FIRE:   17 18:14:54    -2476.954906        0.121700
FIRE:   18 18:

In [ ]:
# Save relaxed structure
relaxed_structure = result["final_structure"]
relaxed_structure.to(filename=output_path)

print(f"✅ Relaxed structure saved to: {output_path}")

✅ Relaxed structure saved to: /home/mehuldarak/summer/relax_final/cellrelaxed_LLZO_010_Li_order0_off__Li_100_slab_heavy.cif


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/pymatgen/io/cif.py:1724: FutureWarning: We strongly discourage using implicit binary/text `mode`, and this would not be allowed after 2025-06-01. I.e. you should pass t/b in `mode`.
  with zopen(filename, mode=mode) as file:


: 